# tflite custom operator

- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/custom_operators.md

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
print('TensorFlow Version: %s' % tf.__version__)

TensorFlow Version: 1.15.0


## Input data

In [3]:
np_x = np.array(
    [-0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], 
    dtype=np.float32, ndmin=2)
np_y_ = np.array(
    [-0.7754, -0.6435, -0.5236, -0.4115, -0.3047, -0.2014, -0.1002, 0.0000, 
     0.1002, 0.2014, 0.3047, 0.4115, 0.5236, 0.6435, 0.7754, 0.9273, 1.1198], 
    dtype=np.float32, ndmin=2)

## Model

- y = arcsin(x + 0.2)

In [4]:
offset_initializer = tf.constant_initializer(value=0.0)

In [5]:
offset = tf.get_variable("offset", [1,], tf.float32, initializer=offset_initializer)
x = tf.placeholder(tf.float32, shape=np_x.shape)
y = tf.asin(x + offset)
y_ = tf.placeholder(tf.float32, shape=np_y_.shape)
loss = tf.reduce_sum(tf.square(y - y_))

## Fit & convert to tflite model

- https://www.tensorflow.org/lite/convert/python_api

In [6]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(500):
        sess.run(train, feed_dict={x: np_x, y_: np_y_})
    
    l, o = sess.run( (loss, offset), feed_dict={x: np_x, y_: np_y_})
    
    converter = tf.lite.TFLiteConverter.from_session(sess, [x], [y])
    converter.allow_custom_ops=True
    tflite_model = converter.convert()
    
open("tflite_op_sine.tflite", "wb").write(tflite_model)

W1025 16:07:53.709619 16480 deprecation.py:323] From D:\Program\miniconda3\envs\tf\lib\site-packages\tensorflow_core\lite\python\util.py:249: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W1025 16:07:53.710589 16480 deprecation.py:323] From D:\Program\miniconda3\envs\tf\lib\site-packages\tensorflow_core\python\framework\graph_util_impl.py:277: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


712

## Show result

In [8]:
print('\t loss =   %E' % l)
print('\t offset = %.4f' % o)

	 loss =   7.084457E-09
	 offset = 0.2000
